In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

m:\Third Year\Sixth Semester\Projects\NLP-Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import faiss 
import numpy as np
import pickle

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [5]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [6]:
# creating an instance of the class
model = semanticEmbedding()

In [7]:
index = FaissIdx(model)

In [8]:
index.load_index('index.bin', 'doc_map.pkl')

In [43]:
import google.generativeai as genai

In [44]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [45]:
model = genai.GenerativeModel('gemini-pro')

In [63]:
query = "I want some money for my startup"

In [64]:
query_result = index.search_doc(query)

In [65]:
context = ''

In [66]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key

In [67]:
context

'Website of Startup India = Startup India is a flagship initiative of the Government of India, intended to build a strong eco-system for nurturing innovation and Startups in the country that will drive sustainable economic growth and generate large scale employment opportunities. The objective is that India must become a nation of job creators instead of being a nation of job seekers. The Government through this initiative aims to empower Startups to grow through innovation and design.Entrepreneurial and Managerial Development of SMEs through Incubators = The objective of the scheme scheme is to provide early stage funding to nurture innovative business ideas (new indigenous technology, processes, products, procedures, etc.) that could be commercialised in a year.Startup India Seed Fund Scheme (SISFS) = The Startup India Seed Fund Scheme aims to provide financial assistance to startups for proof of concept, prototype development, product trials, market entry and commercialisation.Entre

In [68]:
content = ""

In [69]:
if context == '':
    print('Is there anything else I can help you with')
else:
    content = f'Respond like like you are a consultant, Use this query and respond with the following context in a presentatble manner. QUERY:{query} CONTEXT:{context}'

In [70]:
content

'Respond like like you are a consultant, Use this query and respond with the following context in a presentatble manner. QUERY:I want some money for my startup CONTEXT:Website of Startup India = Startup India is a flagship initiative of the Government of India, intended to build a strong eco-system for nurturing innovation and Startups in the country that will drive sustainable economic growth and generate large scale employment opportunities. The objective is that India must become a nation of job creators instead of being a nation of job seekers. The Government through this initiative aims to empower Startups to grow through innovation and design.Entrepreneurial and Managerial Development of SMEs through Incubators = The objective of the scheme scheme is to provide early stage funding to nurture innovative business ideas (new indigenous technology, processes, products, procedures, etc.) that could be commercialised in a year.Startup India Seed Fund Scheme (SISFS) = The Startup India 

In [72]:
response = model.generate_content(content)

In [73]:
print(response.text)

**Presentable Response**

**Subject: Funding Options for Your Startup through Government Initiatives**

**Introduction**

In alignment with your query, we have identified several funding opportunities from the Government of India tailored to support startups like yours. These initiatives aim to foster innovation, drive economic growth, and create employment opportunities.

**Government of India Initiatives for Startup Funding**

**1. Startup India Seed Fund Scheme (SISFS)**

* Provides financial assistance for proof of concept, prototype development, product trials, and commercialization.

**2. Entrepreneurial and Managerial Development of SMEs through Incubators**

* Offers early-stage funding for innovative business ideas with a focus on commercialization.

**3. Entrepreneurial Schemes of National Backward Classes Finance and Development (NBCFDC)**

* Provides financial support to entrepreneurs from backward classes through various schemes, such as Mahila Samriddhi Yojana and Krishi 